# Data Collection 

We will look at financial data from yfinance and FRED APIs. The aim is for weekly frequency and about 10 years out, but these can be changed later. The list is as follows:
- $r^{SPY}_t$ (weekly log return, adjusted close)
- $r^{XLF}_t$ (weekly log return, adjusted close)
- $\Delta \log(VIX_t)$ (weekly change in log VIX)
- $\Delta y^{10Y}_t$ (weekly change in 10Y yield)
- $\Delta(BAA_t - AAA_t)$ (weekly change in credit spread)

In [1]:
import warnings
warnings.filterwarnings("ignore")

from dataclasses import dataclass
from typing import Optional, Dict

import numpy as np
import pandas as pd

import yfinance as yf
from pandas_datareader import data as pdr  # FRED (no API key required)

# Optional: nicer progress prints
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)

@dataclass
class Config:
    start: str = "2005-01-01"
    end: Optional[str] = None         # None -> today
    freq: str = "W-FRI"               # "W-FRI" (weekly), "B" (business daily), "D" (daily)
    ff_fred: bool = True              # forward-fill FRED series before resampling
    price_field: str = "Adj Close"    # for ETFs: "Adj Close" is best

cfg = Config()
print(cfg)

Config(start='2005-01-01', end=None, freq='W-FRI', ff_fred=True, price_field='Adj Close')


In [2]:
def _to_freq(df: pd.DataFrame, freq: str, how: str = "last") -> pd.DataFrame:
    """
    Resample to target frequency.
    - For prices: use last observation in period (how="last").
    - For rates/spreads: also usually last; then later we difference.
    """
    if freq in ("D", "B"):  # already daily-ish; just align to business days if needed
        if freq == "B":
            df = df.asfreq("B")
        return df

    # weekly / monthly / etc.
    if how == "last":
        return df.resample(freq).last()
    elif how == "mean":
        return df.resample(freq).mean()
    else:
        raise ValueError("how must be 'last' or 'mean'")

def fetch_yahoo_prices(tickers, start, end, freq, price_field="Adj Close") -> pd.DataFrame:
    raw = yf.download(
        tickers=tickers,
        start=start,
        end=end,
        auto_adjust=False,
        progress=False,
        group_by="column",
    )
    # yfinance returns multi-index columns when multiple tickers
    if isinstance(raw.columns, pd.MultiIndex):
        px = raw[price_field].copy()
    else:
        px = raw[[price_field]].rename(columns={price_field: tickers[0]}).copy()

    px.index = pd.to_datetime(px.index)
    px = px.sort_index()
    px = _to_freq(px, freq=freq, how="last")
    return px

def fetch_fred(series_map: Dict[str, str], start, end, freq, ff_fred=True) -> pd.DataFrame:
    """
    series_map: {"dgs10": "DGS10", "baa": "BAA", "aaa": "AAA", "vix_fred": "VIXCLS", ...}
    """
    out = {}
    for col, fred_id in series_map.items():
        s = pdr.DataReader(fred_id, "fred", start=start, end=end)
        s.columns = [col]
        out[col] = s

    df = pd.concat(out.values(), axis=1).sort_index()

    # FRED series often missing weekends/holidays. Forward-fill can help before resampling.
    if ff_fred:
        df = df.ffill()

    df = _to_freq(df, freq=freq, how="last")
    return df

In [3]:
# Adjustable inputs 
cfg.start = "2005-01-01"
cfg.end = None           # or e.g. "2025-12-31"
cfg.freq = "W-FRI"       # try "B" for business daily, or "W-FRI" for weekly

# Yahoo tickers 
yahoo_tickers = ["SPY", "XLF", "^VIX"]   # swap XLF<->XLE/IWM/etc if you want
px = fetch_yahoo_prices(
    tickers=yahoo_tickers,
    start=cfg.start,
    end=cfg.end,
    freq=cfg.freq,
    price_field=cfg.price_field
)

# FRED series 
fred_series = {
    "dgs10": "DGS10",  # 10-year Treasury yield (%)
    "baa":   "BAA",    # Moody's Baa corporate yield (%)
    "aaa":   "AAA",    # Moody's Aaa corporate yield (%)
    # Optional if you ever want it: trade-weighted dollar index
    # "dxy": "DTWEXBGS"
}
fred = fetch_fred(
    series_map=fred_series,
    start=cfg.start,
    end=cfg.end,
    freq=cfg.freq,
    ff_fred=cfg.ff_fred
)

In [6]:
px.tail()

Ticker,SPY,XLF,^VIX
Date,,,
2026-01-23,689.229980,53.070000,16.090000
2026-01-30,691.969971,53.439999,17.440001
2026-02-06,690.619995,54.259998,20.370001
2026-02-13,681.750000,51.650002,20.600000
2026-02-20,689.419983,52.490002,19.090000


In [7]:
fred.tail()

,dgs10,baa,aaa
DATE,,,
2026-01-23,4.24,5.88,5.34
2026-01-30,4.26,5.88,5.34
2026-02-06,4.22,5.88,5.34
2026-02-13,4.04,5.88,5.34
2026-02-20,4.08,5.88,5.34


In [9]:
# Merge into one raw aligned dataframe 
raw = pd.concat([px, fred], axis=1).dropna(how="all").sort_index()

print("Yahoo (prices) shape:", px.shape)
print("FRED shape:", fred.shape)
print("Merged raw shape:", raw.shape)
display(raw.head(10))

Yahoo (prices) shape: (1103, 3)
FRED shape: (1103, 3)
Merged raw shape: (1103, 6)


,SPY,XLF,^VIX,dgs10,baa,aaa
2005-01-07,80.344261,16.038649,13.49,4.29,6.02,5.36
2005-01-14,80.208595,15.862629,12.43,4.23,6.02,5.36
2005-01-21,79.218185,15.755945,14.36,4.16,6.02,5.36
2005-01-28,79.659096,15.745280,13.24,4.16,6.02,5.36
2005-02-04,81.558487,16.219988,11.21,4.09,5.82,5.20
2005-02-11,81.924820,16.305330,11.43,4.10,5.82,5.20
2005-02-18,81.667023,15.889299,11.18,4.27,5.82,5.20
2005-02-25,82.372551,15.979971,11.49,4.27,5.82,5.20
2005-03-04,83.254410,16.107979,11.94,4.32,6.06,5.40
2005-03-11,81.667023,15.835958,12.80,4.56,6.06,5.40


## Preprocessing

In this chunk we will rename the variables as needed and apply transformations.

- `r_spy`, `r_xlf` as log returns
- `dlog_vix` = $\Delta \log(VIX)$
- `dy10` = $\Delta (DGS10)$
- `d_credit` = $\Delta (BAA−AAA)$

In addition, we will drop missing ness and check econometric validity (correlation and standardization)

In [11]:
df = raw.copy()

# renaming columns
rename_map = {
    "SPY": "spy_px",
    "XLF": "xlf_px",
    "^VIX": "vix",
    "dgs10": "y10",
    "baa": "baa",
    "aaa": "aaa",
}
df = df.rename(columns=rename_map)

# ensure numeric + sorted index
df = df.apply(pd.to_numeric, errors="coerce").sort_index()

# Construct spreads
df["credit_spread"] = df["baa"] - df["aaa"]  # BAA-AAA in percentage points

# Returns (log) for ETFs
df["r_spy"] = np.log(df["spy_px"]).diff()
df["r_xlf"] = np.log(df["xlf_px"]).diff()

# delta log VIX
df["dlog_vix"] = np.log(df["vix"]).diff()

# delta 10Y yield (percentage points, weekly change)
df["dy10"] = df["y10"].diff()

# delta credit spread
df["d_credit"] = df["credit_spread"].diff()

# Final modeling dataframe (our state space vector) 
df_model = df[["r_spy", "r_xlf", "dlog_vix", "dy10", "d_credit"]].dropna()

print("Model dataframe shape:", df_model.shape)
display(df_model.head(10))

Model dataframe shape: (1102, 5)


,r_spy,r_xlf,dlog_vix,dy10,d_credit
2005-01-14,-0.001690,-0.011035,-0.081836,-0.06,0.00
2005-01-21,-0.012425,-0.006748,0.144334,-0.07,0.00
2005-01-28,0.005550,-0.000677,-0.081204,0.00,0.00
2005-02-04,0.023564,0.029704,-0.166436,-0.07,-0.04
2005-02-11,0.004482,0.005248,0.019435,0.01,0.00
2005-02-18,-0.003152,-0.025846,-0.022115,0.17,0.00
2005-02-25,0.008602,0.005690,0.027351,0.00,0.00
2005-03-04,0.010649,0.007979,0.038417,0.05,0.04
2005-03-11,-0.019251,-0.017032,0.069551,0.24,0.00
2005-03-18,-0.011566,-0.019240,0.026216,-0.05,0.00


In [13]:
display(df_model.describe().T)

,count,mean,std,min,25%,50%,75%,max
r_spy,1102.0,0.001951,0.024590,-0.220564,-0.008954,0.003491,0.014922,0.124800
r_xlf,1102.0,0.001076,0.037185,-0.273899,-0.014318,0.003663,0.017987,0.281596
dlog_vix,1102.0,0.000315,0.148981,-0.556225,-0.085746,-0.009449,0.074983,0.853718
dy10,1102.0,-0.000191,0.118901,-0.520000,-0.080000,-0.010000,0.070000,0.470000
d_credit,1102.0,-0.000109,0.058716,-0.630000,0.000000,0.000000,0.000000,0.940000


#### Summary Statistics

- For our `spy` and `xlf` variables, the mean is 0.1–0.2% weekly which is reasonable and the SD of `xlf` is higher, which makes sense. Large negative values are fine, this is our tail behavior from shock events
- For `dlog_vix`, we see large volatility of volatility (Std = 0.149) along with periods where VIX explodes (Max=0.85) or collapses (Min=-0.56). This is good, since this is the heavy tail behavior we want
- No issues in `dy10`, a 11.9bp change is reasonable by macroeconomic standards
- we see “zero clustering” for weekly credit spreads, which is normal (the 0.00s in quartiles). It just means spreads often don’t move much week-to-week.

In [14]:
display(df_model.corr())

,r_spy,r_xlf,dlog_vix,dy10,d_credit
r_spy,1.000000,0.848492,-0.724654,0.235538,-0.169901
r_xlf,0.848492,1.000000,-0.566692,0.313376,-0.175564
dlog_vix,-0.724654,-0.566692,1.000000,-0.259444,0.064409
dy10,0.235538,0.313376,-0.259444,1.000000,-0.199260
d_credit,-0.169901,-0.175564,0.064409,-0.199260,1.000000


#### Correlations

- The correlation between SPY and XLF is high but acceptable (<0.9). Makes sense since financials co-move with the market. Our shrinkage prior will handle this effectively
- The correlation between SPY and VIX is textbook off-risk structure
- The correlation between SPY and T10Y is 0.23 which is moderate since rising yields often coincide with growth optimism
- credit correlations appear modest (around 0.2)

We will need to standardize before modeling because VIX and yields have 4–6× larger variance than equity returns, which is going to lead to poor mixing in SGLD. We need to preserve the means and SDs since those are used for VaR and ES calculations. The chunks for standardization and export are below

In [17]:
# Keep raw modeling data
data_raw = df_model.copy()

# Compute means and stds
means = data_raw.mean()
stds = data_raw.std()

# Z-score
data_std = (data_raw - means) / stds

print("Standardized dataframe shape:", data_std.shape)
display(data_std.tail())

print("\nMeans (should be ~0):")
print(data_std.mean())

print("\nStds (should be ~1):")
print(data_std.std())

Standardized dataframe shape: (1102, 5)


,r_spy,r_xlf,dlog_vix,dy10,d_credit
2026-01-23,-0.222447,-0.714344,0.094527,0.001603,0.001855
2026-01-30,0.082024,0.157907,0.538683,0.169809,0.001855
2026-02-06,-0.158737,0.380576,1.040280,-0.334810,0.001855
2026-02-13,-0.605007,-1.354640,0.073249,-1.512255,0.001855
2026-02-20,0.375638,0.404905,-0.513097,0.338016,0.001855



Means (should be ~0):
r_spy      -3.223878e-18
r_xlf       8.059695e-18
dlog_vix    2.256715e-17
dy10        0.000000e+00
d_credit    3.223878e-18
dtype: float64

Stds (should be ~1):
r_spy       1.0
r_xlf       1.0
dlog_vix    1.0
dy10        1.0
d_credit    1.0
dtype: float64


In [20]:
raw_export = data_raw.copy()
raw_export.insert(0, "date", raw_export.index)

std_export = data_std.copy()
std_export.insert(0, "date", std_export.index)

# Export
raw_export.to_csv("../data/data_raw.csv", index=False)
std_export.to_csv("../data/data_std.csv", index=False)

print("Files exported:")
print("- data_raw.csv")
print("- data_std.csv")

Files exported:
- data_raw.csv
- data_std.csv
